In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import nltk
from gensim.models import Word2Vec
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer




In [11]:
train = pd.read_csv("train.csv")
train

,comment,label
0,مهبل عاقلين حدك يخدم بلد سويت مادية,1
1,غض البصر عن المحرم ليس بي السهل عن الرجال والن...,1
2,يا لكم من جهلة تدعون العلم أين أنتم من البوصي...,1
3,انا مكبوت كرحلي الكبت نخم باش نسامحك وراس جبهت...,1
4,انهم يحلون في مابينهم واستنساخ ادعية تحريظية ف...,1
...,...,...
11315,ماتو حبيبتي وبقاو هاذو الحثالة ياو فاقو قع هذا...,0
11316,حضو لي حاكم ولاية و بلدية دو راهم يرسكو على جا...,1
11317,الصورة في الوراء من التبوريضة المغربية حتى هي ...,1
11318,اشمن تهريب الزطله انت مالها منين كاتجي من حداك...,0


In [12]:
test = pd.read_csv("test.csv")
test

,comment,label
0,قيل لأحد الصالحين؛ كم بينا وبينعرش الرحمن ؟ ...,0
1,شروق غلطت غلط كبير إعلان حول تواصل إجتماع نسيب...,0
2,عيب وعار أن الرئيس الجزائري يقول إن الصحراء مح...,1
3,على بلحاج راجل و الدولة عميلة فرنسا العدو كل ا...,1
4,لكن في المغرب و له الحمد يوم بعد يوم نتلمس الق...,0
...,...,...
2826,وعلى كل حال الرواية لا تعود لسلفية ولا تعبر عن...,0
2827,مراكش قادر على القطاع نتاعك دير الاستقالة نتاع...,0
2828,لاحول ولا قوة الاباله الهم إن هذا منكر وبدع وش...,0
2829,تمثيل صفر كفاه يحبس فالوطو وخاىج بجري قدامها,0


In [13]:
# Initialize stemmer and lemmatizer
stemmer = SnowballStemmer("arabic")
lemmatizer = WordNetLemmatizer()

# Define a function for text preprocessing
def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Stemming
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    
    # Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]
    
    return ' '.join(lemmatized_tokens)

# Apply text preprocessing to the comments column
train['comment'] = train['comment'].apply(preprocess_text)
test['comment'] = test['comment'].apply(preprocess_text)



In [15]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(train['comment'])
X_test = tfidf_vectorizer.transform(test['comment'])

y_train = train['label']
y_test = test['label']


In [17]:
# Train SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(X_train, y_train)

# Predict on testing set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
print(classification_report(y_test, y_pred ))



              precision    recall  f1-score   support

           0       0.67      0.67      0.67      1270
           1       0.73      0.73      0.73      1561

    accuracy                           0.70      2831
   macro avg       0.70      0.70      0.70      2831
weighted avg       0.70      0.70      0.70      2831



In [20]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

rf_classifier.fit(X_train, y_train)
# Predict on testing set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
print(classification_report(y_test, y_pred ))

              precision    recall  f1-score   support

           0       0.63      0.67      0.65      1270
           1       0.72      0.68      0.70      1561

    accuracy                           0.68      2831
   macro avg       0.67      0.68      0.67      2831
weighted avg       0.68      0.68      0.68      2831



In [19]:
sgd_classifier = SGDClassifier(loss='log', alpha=0.00001, max_iter=1000, random_state=42)
sgd_classifier.fit(X_train, y_train)

# Predict on test set
predictions = sgd_classifier.predict(X_test)
print(classification_report(y_test,predictions ))


              precision    recall  f1-score   support

           0       0.67      0.66      0.66      1270
           1       0.72      0.74      0.73      1561

    accuracy                           0.70      2831
   macro avg       0.70      0.70      0.70      2831
weighted avg       0.70      0.70      0.70      2831



C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
